# 워크플로우 구성 - Chains & Memory (체인과 메모리)

**LCEL(LangChain Expression Language)**을 사용하여 여러 작업을 연결하고, 복잡한 프로세스를 처리합니다.  
또한 대화의 맥락(Context)을 기억하는 **Memory** 기능도 실습합니다.

**학습 목표:**
1. **Simple Chain:** 프롬프트, 모델, 파서를 연결하여 기본적인 LLM 파이프라인 만들기
2. **Sequential Chain:** 하나의 체인 출력을 다음 체인의 입력으로 연결하여 순차적 작업 수행하기
3. **Router Chain (Conditional):** 입력 내용에 따라 적절한 체인을 선택하여 실행하는 분기 로직 구현하기
4. **Memory:** 대화 내역(History)을 기억하고 문맥을 이해하는 챗봇 체인 만들기

### 1. 환경 설정 (Environment Setup)

In [ ]:
%pip install langchain langchain-openai langchain-community -Uq

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")

# LangSmith 설정 (선택)
os.environ["LANGSMITH_TRACING"] = 'true'
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")

---### 2. Simple Chain (기본 체인)

가장 기본적인 LCEL 패턴입니다.
`Prompt(입력 구성) -> Model(생성) -> Parser(후처리)` 순서로 연결합니다.

```python
chain = prompt | model | output_parser
```

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 구성 요소 준비
prompt = PromptTemplate.from_template('{country}의 수도는 어디인가요?')
llm = ChatOpenAI(model='gpt-4o-mini')
output_parser = StrOutputParser()

# 2. 체인 연결 (Pipe 연산자 사용)
chain = prompt | llm | output_parser

# 3. 실행
print(chain.invoke({'country': '대한민국'}))

---### 3. Sequential Chain (순차 체인)

하나의 체인 실행 결과를 다음 체인의 입력으로 넘겨줍니다.
예시: `영어 문장 번역` -> `번역된 문장 요약`

In [ ]:
# 첫 번째 체인: 영어 -> 한글 번역
prompt1 = PromptTemplate.from_template('다음 내용을 한글로 번역하세요.\n\n{eng_text}')
chain1 = prompt1 | llm | StrOutputParser()

# 두 번째 체인: 한글 -> 요약
# 입력 변수명이 {kor_text}로 되어있음. chain1의 출력을 여기에 매핑해줘야 함.
prompt2 = PromptTemplate.from_template('다음 내용을 한 문장으로 요약하세요.\n\n{kor_text}')
chain2 = prompt2 | llm | StrOutputParser()

# 전체 체인 연결
# chain1의 출력(문자열)이 chain2의 첫 번째 인자(kor_text)로 자동 전달됨 (인자가 1개일 때)
full_chain = chain1 | chain2

eng_input = """
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). 
You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. 
LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""

print("[원본]:", eng_input.strip()[:50], "...")
result = full_chain.invoke({'eng_text': eng_input})
print("\n[요약 결과]:", result)

---### 4. Router Chain (분기 체인)

입력된 질문의 **주제**나 **의도**에 따라 다른 체인을 실행하고 싶을 때 사용합니다.
예: 수학 문제는 `Math Chain`, 일상 대화는 `Chat Chain`으로 분기.

- `RunnableBranch`: 조건에 따라 분기 처리

In [ ]:
from langchain_core.runnables import RunnableBranch

# 1. 분기될 체인들 정의
# 수학 문제용 체인
math_chain = (
    PromptTemplate.from_template("다음 수식을 단계별로 풀이해 주세요:\n{question}")
    | llm
    | StrOutputParser()
)

# 일반 대화용 체인
general_chain = (
    PromptTemplate.from_template("친절하게 답변해 주세요:\n{question}")
    | llm
    | StrOutputParser()
)

# 2. 분기 조건 함수
def is_math_question(input_dict: dict) -> bool:
    q = input_dict.get('question', '')
    return any(keyword in q for keyword in ['계산', '수식', '더하기', '곱하기', '+', '*', 'calc'])

# 3. Branch 연결
branch_chain = RunnableBranch(
    (is_math_question, math_chain),  # (조건 함수, 실행할 체인)
    general_chain                    # 조건이 안 맞을 때 실행할 기본 체인
)

# 테스트
print("1. 수학 문제:")
print(branch_chain.invoke({'question': '125 * 3 + 50 계산해줘'}))

print("\n2. 일반 대화:")
print(branch_chain.invoke({'question': '오늘 점심 메뉴 추천해줘'}))

---### 5. Memory With Chain (대화 맥락 유지)

LLM은 기본적으로 **무상태(Stateless)**입니다. 이전 대화 내용을 기억하지 못합니다.
대화 내역을 저장하고 프롬프트에 주입해주는 `RunnableWithMessageHistory`를 사용하여 기억을 구현합니다.

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 1. 메모리 저장소 (세션 ID별로 대화 기록 저장)
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 2. 프롬프트 정의 (MessagesPlaceholder로 히스토리 들어갈 자리 마련)
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 챗봇입니다."),
    MessagesPlaceholder(variable_name="history"),  # 대화 내역이 여기에 주입됨
    ("human", "{question}")
])

# 3. 체인 생성
chain = prompt | llm | StrOutputParser()

# 4. 히스토리 기능 래핑
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

# 5. 대화 실행 (세션 ID: user1)
config = {"configurable": {"session_id": "user1"}}

response1 = chain_with_history.invoke(
    {"question": "내 이름은 김철수야. 기억해줘."}, 
    config=config
)
print("AI:", response1)

response2 = chain_with_history.invoke(
    {"question": "내 이름이 모니?"}, 
    config=config
)
print("AI:", response2)

# 6. 다른 세션 (세션 ID: user2) - 기억 못함 확인
config2 = {"configurable": {"session_id": "user2"}}
response3 = chain_with_history.invoke(
    {"question": "내 이름이 모니?"}, 
    config=config2
)
print("AI (user2):", response3)